In [1]:
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture
from hmmlearn import hmm
import pickle
import gzip
# from MLP import MLP
import warnings
# from test_mlp import MLP
# from hmm_dnn import *

In [2]:
data = pd.read_hdf('./features/mfcc/timit.hdf')
data

,features,labels
0,"[6.637863509448256, -23.904281634027743, -7.55...",sil
1,"[6.610915477498382, -25.73031169552281, -5.509...",sil
2,"[6.771937066881559, -27.963432899055178, -3.20...",sil
3,"[6.872044929365073, -32.173795447144215, -4.17...",sil
4,"[6.730539804480821, -29.867820505219058, -3.68...",sil
...,...,...
1236538,"[7.411218215305723, -25.98452069846578, -10.31...",er
1236539,"[7.1910167287501805, -29.71583750314248, -14.2...",er
1236540,"[7.027595436547739, -27.188134257858362, -11.8...",er
1236541,"[7.038284964885829, -28.178316295879007, -11.8...",er


In [3]:
features = np.array(data['features']).tolist()
labels = np.array(data['labels']).tolist()

In [4]:
np.shape(features)

(1236543, 13)

In [5]:
phonemes = set(labels)
phonemes

{'',
 'aa',
 'ae',
 'ah',
 'aw',
 'ay',
 'b',
 'ch',
 'd',
 'dh',
 'dx',
 'eh',
 'er',
 'ey',
 'f',
 'g',
 'hh',
 'ih',
 'iy',
 'jh',
 'k',
 'l',
 'm',
 'n',
 'ng',
 'ow',
 'oy',
 'p',
 'r',
 's',
 'sh',
 'sil',
 't',
 'th',
 'uh',
 'uw',
 'v',
 'w',
 'y',
 'z'}

In [6]:
len(phonemes)

40

In [7]:
phoneme_dict = {}
count = 0
for phoneme in phonemes:
    phoneme_dict[count] = phoneme
    count = count + 1
 
phoneme_dict

    

{0: '',
 1: 'ah',
 2: 'f',
 3: 'l',
 4: 'k',
 5: 'er',
 6: 'd',
 7: 'b',
 8: 'g',
 9: 'r',
 10: 'uh',
 11: 'ih',
 12: 'uw',
 13: 'eh',
 14: 'dx',
 15: 's',
 16: 'ow',
 17: 'aw',
 18: 'dh',
 19: 'n',
 20: 'sh',
 21: 'hh',
 22: 'ae',
 23: 'sil',
 24: 'oy',
 25: 'ch',
 26: 'aa',
 27: 'ay',
 28: 'iy',
 29: 'w',
 30: 'y',
 31: 't',
 32: 'z',
 33: 'th',
 34: 'ng',
 35: 'm',
 36: 'ey',
 37: 'p',
 38: 'jh',
 39: 'v'}

In [8]:
data['labels'].value_counts()

sil    207051
ih      84082
n       67327
l       60397
s       55543
iy      52096
r       51174
er      47657
ah      39071
k       37382
t       35051
m       32820
d       32575
aa      31440
        31047
z       28804
w       26248
dh      21745
ae      20984
eh      20609
hh      20103
b       19302
dx      19008
p       18661
f       17063
uw      16135
ay      15634
ey      14811
sh      14774
v       14714
g       13894
y       12781
ow      12437
jh      10187
ng       9805
ch       7426
th       5495
aw       4318
oy       3980
uh       2912
Name: labels, dtype: int64

In [9]:
data[data['labels'] == 'oy']

,features,labels
314,"[14.47224666101531, 13.853338018923196, -7.975...",oy
315,"[14.489470093622852, 17.273267592967724, -5.11...",oy
316,"[14.480066058960855, 11.07302898442892, -1.370...",oy
317,"[14.252990386692453, 9.306273854461965, 4.8608...",oy
5271,"[14.933248279581331, 4.288542170953272, -16.46...",oy
...,...,...
1235212,"[8.503101658299988, -0.017400129084964362, 17....",oy
1235213,"[7.980899824347693, -1.4408768712920004, 20.98...",oy
1235214,"[7.43729188754439, -4.9276528950042815, 15.262...",oy
1235215,"[7.106311640601912, -9.340361282696007, 13.794...",oy


In [10]:
data[data['labels'] == 'ih']

,features,labels
69,"[16.387267863620163, -50.47940073762711, 7.847...",ih
70,"[16.93315001404509, -47.5162330114518, 14.1328...",ih
71,"[17.20032095152797, -41.421475353940096, 18.51...",ih
72,"[17.270541444587533, -44.95154488581582, 17.18...",ih
73,"[17.116937929094373, -38.20076036641038, 19.80...",ih
...,...,...
1236319,"[6.617694655540228, -22.20273518014949, 0.4793...",ih
1236320,"[6.650880821368861, -22.95487524322874, 0.5020...",ih
1236321,"[6.6854883796732, -31.0694302951681, -2.711983...",ih
1236322,"[6.690360152529087, -30.535767030896174, -2.28...",ih


# Training GMM-HMM model for 'uh' phoneme

In [11]:
uh_feature_list = list(data[data['labels'] == 'uh']['features'])
uh_feature_list

[array([ 13.27078133,  -2.30543951, -25.40747065,  -4.29418289,
          0.63329917, -23.74610057,  -7.37471844, -13.64706659,
        -23.23845068,   6.17005434,  14.77663796, -17.70727146,
          0.43969521]),
 array([ 10.77467321, -29.81708816,   1.03099375,  24.28869296,
        -12.84591881, -14.97405267,   2.93015381, -11.93435606,
        -20.18262006, -16.62115931,   3.68343828, -18.90142843,
        -18.33361092]),
 array([ 8.54766449e+00, -2.91780527e+01,  1.77380913e+00,  2.03828738e+01,
         1.31258877e-02, -1.34712713e+01, -7.45006346e+00, -2.01223457e+01,
        -1.23449418e+01, -3.49354337e+00, -2.62750795e+00, -1.46041909e+01,
        -7.74811732e+00]),
 array([  8.27491996, -31.45004886,   1.16685006,  13.75840193,
          0.46102019, -10.05550796,  -4.64965718, -12.13906284,
         -4.3580057 ,  -0.6777642 ,  -3.04373072,  -8.91364801,
         -5.47190755]),
 array([  8.78767947, -33.59487492,  -4.20191237,   9.04150445,
         -0.05122668,   2.4282067

In [12]:
np.mean(uh_feature_list,axis = 0)

array([ 11.1837323 , -14.32947974,  -2.87605953,  -5.91570163,
        -8.42580469,  -5.04375995,  -6.93277695,  -9.54501634,
        -4.65527874,  -6.07570178,  -2.82893865,  -4.50870686,
        -4.73518302])

In [13]:
np.std(uh_feature_list,axis=0)

array([ 3.80445837, 18.1163937 , 13.36795235, 13.66432654, 16.84425162,
       13.14526911, 13.16651582, 14.0346824 , 12.99729157, 11.23050256,
       10.65778605, 10.85120529,  9.33056963])

# Normalize 'uh' features with 0 mean and 1 std

In [14]:
temp_data = list()
for j in range(len(uh_feature_list)):
    temp_data.append(np.array(uh_feature_list[j]))
        
        
uh_train_list = list()
for j in range(len(temp_data)):
    uh_train_list.append(((temp_data[j] - np.mean(temp_data, axis=0)) / np.std(temp_data, axis=0)).tolist())




In [15]:
uh_train_list

[[0.5485798073083146,
  0.6637104725543637,
  -1.6854796104305234,
  0.1186680315646543,
  0.5378157524686465,
  -1.4227430762776176,
  -0.03356556121393924,
  -0.29227952165949994,
  -1.4297726449475594,
  1.0904014368717918,
  1.6518981078966706,
  -1.2163224494400935,
  0.5546154668002042],
 [-0.1075209778563601,
  -0.8548946702956847,
  0.29227013803197355,
  2.21045614715471,
  -0.26241083464032144,
  -0.755427114535426,
  0.7490919309493692,
  -0.17024537172285026,
  -1.1946597675818225,
  -0.9390013914709175,
  0.611044066458546,
  -1.3263707741378477,
  -1.4574059717170909],
 [-0.6928891186803364,
  -0.8196207905819707,
  0.3478370163713055,
  1.9246155549582706,
  0.500997655825662,
  -0.6411060328452327,
  -0.03928803317562132,
  -0.7536564798696128,
  -0.5916358081538429,
  0.2299236737696027,
  0.018899863731894378,
  -0.9303560107696884,
  -0.3229100062707785],
 [-0.764579884439864,
  -0.9450318535486995,
  0.30243297410968495,
  1.439815091020877,
  0.5275879914733536,
  

In [16]:
np.mean(uh_train_list,axis=0)

array([-9.17916536e-16,  3.27958051e-16, -2.48122646e-16, -1.76141153e-17,
       -2.66118019e-17, -6.54047939e-16,  3.85871127e-16,  8.63072587e-17,
       -6.36700704e-17,  3.64272867e-16, -4.11758539e-18, -1.11052803e-15,
       -8.79333236e-16])

In [17]:
np.std(uh_train_list,axis=0)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [18]:
gmmhmm_module_list = list()

In [19]:
gmmhmm_module_list.append(hmm.GMMHMM(n_components=3,n_mix=64,covariance_type='diag',random_state=42,verbose=True,n_iter=300,tol=1e-5))

In [20]:
for i, module in enumerate(gmmhmm_module_list):
    module.fit(uh_train_list)

         1      -53655.8273             +nan
         2      -40236.5164      +13419.3110
         3      -35190.3660       +5046.1503
         4      -32921.2530       +2269.1130
         5      -31725.1706       +1196.0824
         6      -31001.1563        +724.0143
         7      -30567.5593        +433.5970
         8      -30343.4203        +224.1390
         9      -30220.6881        +122.7322
        10      -30117.8040        +102.8841
        11      -30038.1720         +79.6320
        12      -29970.3042         +67.8678
        13      -29911.9396         +58.3646
        14      -29862.9181         +49.0215
        15      -29827.7739         +35.1442
        16      -29793.4582         +34.3157
        17      -29762.5181         +30.9401
        18      -29744.8556         +17.6624
        19      -29734.4341         +10.4215
        20      -29725.5144          +8.9197
        21      -29717.0060          +8.5084
        22      -29702.0564         +14.9496
        23

       184      -29539.3878          +0.0000
       185      -29539.3878          +0.0000
       186      -29539.3878          +0.0000
       187      -29539.3878          +0.0000
       188      -29539.3878          +0.0000
       189      -29539.3878          +0.0000
       190      -29539.3878          +0.0000
       191      -29539.3877          +0.0000
       192      -29539.3877          +0.0000


In [21]:
for i, module in enumerate(gmmhmm_module_list):
    print(module.transmat_)
    print()
    


[[0.82094923 0.12706593 0.05198483]
 [0.06487527 0.78571224 0.14941249]
 [0.04037902 0.08639113 0.87322985]]



In [22]:
for i, module in enumerate(gmmhmm_module_list):
    print(module.means_)
    print(module.means_.shape)

[[[ 0.91999796  1.18009321  0.72489118 ... -0.37059956  0.1905807
   -0.27655348]
  [ 0.37043431  0.41480577 -1.24363053 ...  1.87150045  1.15427858
    0.39364959]
  [-0.22575127  0.37961825 -0.57919447 ... -1.03895392 -0.35493458
   -0.74396294]
  ...
  [ 0.82773369  1.71504887 -1.11965795 ... -0.42215544  0.26533049
    1.61513875]
  [ 0.87814191  1.53702927 -0.16062849 ...  1.81745363  1.30748188
    0.63200565]
  [ 0.4817213   1.28936297 -1.64559951 ...  1.42706601  0.41212359
   -0.20717543]]

 [[-0.4139356   0.1149253   1.57064855 ... -1.33466815 -2.22947043
    0.59195159]
  [-0.9140802   0.62060241  1.09822378 ... -0.19811943 -0.90914087
   -0.76581672]
  [ 0.49603955  0.75499244  0.81352466 ...  0.4211459  -0.35007825
    0.38505059]
  ...
  [-0.84485829  0.43248692  0.9698729  ... -1.57418759 -1.31211165
   -0.35777715]
  [ 0.06556008  0.81690677  0.57849782 ...  1.79446974 -1.13191235
    1.02384197]
  [-0.07391723  0.97926451 -0.45533242 ... -0.71910778 -0.47542226
   -0.8

In [23]:
for i, module in enumerate(gmmhmm_module_list):
    print(module.covars_)
    print(module.covars_.shape)

[[[1.89726606e-02 4.69330410e-02 8.22039039e-02 ... 1.00927487e+00
   4.89902391e-01 9.01298613e-02]
  [7.33363801e-02 1.99399365e-01 2.18287380e-01 ... 3.50511767e-01
   2.15801402e-01 1.65762764e+00]
  [5.31034503e-02 1.60404699e-02 7.17216453e-02 ... 6.64192439e-01
   1.09222300e+00 9.13401240e-01]
  ...
  [8.04900675e-02 1.12110036e-01 6.01695606e-02 ... 2.53134782e-01
   4.33502842e-01 9.12627514e-01]
  [7.91780645e-02 6.91855978e-02 2.26821491e-01 ... 3.52003236e-01
   1.53935262e-01 1.46915832e-01]
  [6.06899998e-03 5.51619292e-03 7.35672914e-02 ... 1.18433924e-02
   1.28315575e-01 1.11018752e-01]]

 [[9.10429338e-02 2.92099393e-01 1.19014397e-01 ... 1.90987418e-01
   2.42943039e-02 1.88728574e-01]
  [1.46007487e-02 5.75177345e-02 3.62215399e-02 ... 4.24017325e-01
   4.51936483e-01 4.02345637e-01]
  [2.90280871e-02 6.96416099e-02 2.70449907e-01 ... 2.17247859e-01
   2.75355398e-01 7.35704642e-01]
  ...
  [3.43089540e-02 1.17451241e-01 1.18754125e-01 ... 3.16776866e-01
   6.03040

In [24]:
for i, module in enumerate(gmmhmm_module_list):
    print(module.weights_)
    print(module.weights_.shape)

[[0.029571   0.00932811 0.02262849 0.00933208 0.03139983 0.01089153
  0.00932882 0.03142031 0.02310049 0.00934065 0.01537456 0.03109204
  0.02836995 0.02293832 0.02171647 0.01867994 0.00933095 0.01244317
  0.01239306 0.02021716 0.01399813 0.03075706 0.0031107  0.01244278
  0.04728671 0.00777668 0.00622139 0.01866413 0.01712117 0.0124426
  0.00932853 0.01244312 0.01710016 0.00933191 0.01699206 0.02686871
  0.01088189 0.00777674 0.00777674 0.00942805 0.01241654 0.0123304
  0.01242909 0.00933095 0.0155555  0.02020998 0.01244159 0.01554107
  0.01244268 0.00777674 0.00933208 0.01703784 0.05136028 0.01394738
  0.00777674 0.01246112 0.01863678 0.01398539 0.0031107  0.01260085
  0.00622123 0.00777614 0.01241133 0.00622139]
 [0.0065027  0.04914646 0.00758824 0.04598449 0.00542862 0.01458477
  0.02073237 0.00542863 0.02775576 0.02206034 0.02355224 0.01871398
  0.0032571  0.02445564 0.01194042 0.0885335  0.01302608 0.01694873
  0.01752289 0.00649822 0.02779941 0.01450366 0.0108504  0.00760012
  0

In [25]:
uh_train_list[0]

[0.5485798073083146,
 0.6637104725543637,
 -1.6854796104305234,
 0.1186680315646543,
 0.5378157524686465,
 -1.4227430762776176,
 -0.03356556121393924,
 -0.29227952165949994,
 -1.4297726449475594,
 1.0904014368717918,
 1.6518981078966706,
 -1.2163224494400935,
 0.5546154668002042]

In [26]:
prob, path = module.decode(np.array(uh_train_list[0]).reshape(1,-1))
print(f" path of uh is {path} and probability is {prob}")

 path of uh is [0] and probability is -12.805740125885498


In [27]:
np.array(uh_train_list)

array([[ 0.54857981,  0.66371047, -1.68547961, ...,  1.65189811,
        -1.21632245,  0.55461547],
       [-0.10752098, -0.85489467,  0.29227014, ...,  0.61104407,
        -1.32637077, -1.45740597],
       [-0.69288912, -0.81962079,  0.34783702, ...,  0.01889986,
        -0.93035601, -0.32291001],
       ...,
       [-1.15560746,  0.20256024,  1.4295919 , ..., -1.29409897,
        -0.283016  , -0.95724904],
       [-1.16258076,  0.05581121,  1.14150238, ..., -2.4839669 ,
        -2.40867286, -2.31570041],
       [-1.27422478,  0.26945752,  1.30411541, ..., -2.15263877,
        -2.38354605, -1.79740752]])

In [28]:
_, path_uh = module.decode(np.array(uh_train_list))
print(f" path of uh is {path_uh}  and path length is {len(path_uh)}")

 path of uh is [0 1 2 ... 1 1 1]  and path length is 2912


In [29]:
for i, module in enumerate(gmmhmm_module_list):
    print(module.sample(10))

(array([[ 0.46941413,  1.1588867 ,  0.19106543, -0.03056244, -3.24757375,
        -0.31332599,  1.65746968, -0.01610503, -0.45565952, -1.91141027,
         2.05399247, -0.16125713,  2.05905916],
       [ 1.06917732,  1.62695872, -2.67996638, -1.20304277, -0.58474014,
         1.0495421 , -1.40993632, -2.65079207,  0.36404541,  1.48558184,
        -1.12352228,  1.1014134 ,  1.04874829],
       [ 0.42731502,  1.20791724, -0.11740264, -1.57704936, -2.93152583,
        -1.17745656,  0.91971195, -0.40124281, -0.53830815,  0.15203296,
        -0.62027794,  0.02609131,  1.28238997],
       [-1.05726693,  0.35860737,  1.15971047,  0.73745275,  1.25075352,
        -0.10507998, -0.29892149, -0.23100386, -0.42822558, -0.56813228,
        -1.3088421 , -0.32541039, -0.5292096 ],
       [-1.27674355,  0.45295657,  1.13887512,  1.03492431,  0.75037068,
        -0.16203156,  0.51516854, -0.22606485, -0.47045602, -1.07686812,
        -0.5983801 , -1.12671933, -0.60547139],
       [ 1.00283962,  1.93329

In [30]:
phonetic_uh = np.array(uh_train_list)
phonetic_uh

array([[ 0.54857981,  0.66371047, -1.68547961, ...,  1.65189811,
        -1.21632245,  0.55461547],
       [-0.10752098, -0.85489467,  0.29227014, ...,  0.61104407,
        -1.32637077, -1.45740597],
       [-0.69288912, -0.81962079,  0.34783702, ...,  0.01889986,
        -0.93035601, -0.32291001],
       ...,
       [-1.15560746,  0.20256024,  1.4295919 , ..., -1.29409897,
        -0.283016  , -0.95724904],
       [-1.16258076,  0.05581121,  1.14150238, ..., -2.4839669 ,
        -2.40867286, -2.31570041],
       [-1.27422478,  0.26945752,  1.30411541, ..., -2.15263877,
        -2.38354605, -1.79740752]])

In [31]:
len(phonetic_uh)
num_batches = int(len(phonetic_uh)/32)
num_batches

91

In [32]:
phonetic_uh_label = np.array(path_uh).reshape(-1,1)
phonetic_uh_label.shape

(2912, 1)

In [33]:
# print('---- training DNN network for uh features')
# dnn_module_list = list()

# dnn_module_list.append(MLP(13, 3))

# for i, mod in enumerate(dnn_module_list):
#     with warnings.catch_warnings():
#         warnings.simplefilter("ignore")

#         mod.train(phonetic_uh,phonetic_uh_label,epoch=1,batch_size=32)

# Training GMM-HMM for 'oy' features

In [34]:
oy_feature_list = list(data[data['labels'] == 'oy']['features'])
oy_feature_list

[array([ 14.47224666,  13.85333802,  -7.97519911, -28.29456904,
        -47.48191971, -26.57489342,   6.63348615,  -3.59678746,
        -19.24761496,  -7.59537173,   0.74830383,  -4.84687249,
          9.46269097]),
 array([ 14.48947009,  17.27326759,  -5.11584397, -21.60001005,
        -46.07727402, -23.89101209,   9.09833908,  -1.33854679,
         -9.74160865,  -7.3196129 ,  -1.92542505,  -7.87828003,
         17.09473577]),
 array([ 14.48006606,  11.07302898,  -1.37036173, -25.33920522,
        -40.72455623, -13.3849256 ,   3.50239663,   2.89780548,
        -15.84434238,  -2.53639017,  -5.22636908,   5.22011465,
         10.81132854]),
 array([ 14.25299039,   9.30627385,   4.86080399, -27.38687369,
        -51.3422138 , -15.63023014,   9.84864262,  10.70721674,
        -24.96547964,  -8.47132968,   1.77302581,  10.37767123,
         11.69338   ]),
 array([ 1.49332483e+01,  4.28854217e+00, -1.64645715e+01, -1.46843238e+01,
        -6.07678196e+01, -9.40563413e+00,  4.80047343e-03, -

# Normalize 'oy' features with 0 mean and 1 std.

In [35]:
temp_data = list()
for j in range(len(oy_feature_list)):
    temp_data.append(np.array(oy_feature_list[j]))
        
        
oy_train_list = list()
for j in range(len(temp_data)):
    oy_train_list.append(((temp_data[j] - np.mean(temp_data, axis=0)) / np.std(temp_data, axis=0)).tolist())

In [36]:
oy_train_list

[[0.023988942095488845,
  1.0850840283301109,
  -0.06729687025237849,
  -1.5029809602223028,
  -1.3998966836400526,
  -1.1179639040915905,
  0.960972840110492,
  0.5244856924306235,
  -0.9036159254036603,
  -0.32243670307649036,
  -0.12009294362815463,
  -0.27353591144093975,
  1.2290532955874396],
 [0.030934654046501357,
  1.258390674953656,
  0.14204659540358783,
  -1.042618370661909,
  -1.320378897436126,
  -0.9436292482120716,
  1.1210735285759286,
  0.6609330610863984,
  -0.30770851368289526,
  -0.30269887939070533,
  -0.3214482783876871,
  -0.5044420333775969,
  1.9032279486867012],
 [0.02714227900559515,
  0.9441904733860825,
  0.4162665488915806,
  -1.2997504048791537,
  -1.017358516856935,
  -0.261194091582174,
  0.7575977903874822,
  0.9169018461308458,
  -0.6902734039633953,
  0.03966697935350535,
  -0.5700384382169782,
  0.4932791514195331,
  1.3481848371599994],
 [-0.06443076840007565,
  0.8546592709591836,
  0.8724721489035419,
  -1.4405617658539116,
  -1.6184301146636517

In [37]:
np.mean(oy_train_list,axis=0)

array([-2.74235129e-14, -6.37415860e-16,  1.25025618e-15, -7.55119027e-16,
        1.82868795e-15,  4.08550915e-16,  9.66926148e-16, -1.53559466e-15,
       -2.92228553e-16, -2.07539178e-16, -3.64365154e-16, -5.60690522e-18,
        5.40187660e-16])

In [38]:
np.std(oy_train_list,axis=0)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [39]:
gmmhmm_module_list.append(hmm.GMMHMM(n_components=3,n_mix=64,covariance_type='diag',random_state=42,verbose=True,n_iter=400,tol=1e-5))

In [40]:
module2 = gmmhmm_module_list[1]
module2.fit(oy_train_list)

         1      -73048.8678             +nan
         2      -57172.4498      +15876.4180
         3      -51730.4114       +5442.0384
         4      -49006.4020       +2724.0094
         5      -47359.5103       +1646.8917
         6      -46226.6415       +1132.8688
         7      -45520.5164        +706.1251
         8      -40592.3760       +4928.1404
         9      -40331.8673        +260.5087
        10      -40136.6439        +195.2233
        11      -40019.4548        +117.1891
        12      -39931.7225         +87.7323
        13      -39874.8294         +56.8931
        14      -39826.5935         +48.2359
        15      -39800.2838         +26.3096
        16      -39776.8981         +23.3858
        17      -39759.3088         +17.5893
        18      -39747.7417         +11.5671
        19      -39735.7798         +11.9619
        20      -39725.0848         +10.6950
        21      -39716.9419          +8.1430
        22      -39710.9842          +5.9577
        23

       184      -39607.8771          +0.0201
       185      -39607.8568          +0.0203
       186      -39607.8356          +0.0213
       187      -39607.8122          +0.0234
       188      -39607.7850          +0.0272
       189      -39607.7513          +0.0337
       190      -39607.7076          +0.0438
       191      -39607.6511          +0.0565
       192      -39607.5853          +0.0658
       193      -39607.5221          +0.0632
       194      -39607.4709          +0.0512
       195      -39607.4298          +0.0411
       196      -39607.3928          +0.0370
       197      -39607.3562          +0.0366
       198      -39607.3187          +0.0375
       199      -39607.2802          +0.0385
       200      -39607.2410          +0.0392
       201      -39607.2019          +0.0391
       202      -39607.1637          +0.0382
       203      -39607.1274          +0.0363
       204      -39607.0937          +0.0336
       205      -39607.0634          +0.0304
       206

       367      -39590.3960          +0.0025
       368      -39590.3930          +0.0030
       369      -39590.3894          +0.0036
       370      -39590.3852          +0.0042
       371      -39590.3804          +0.0047
       372      -39590.3752          +0.0052
       373      -39590.3697          +0.0055
       374      -39590.3640          +0.0057
       375      -39590.3584          +0.0056
       376      -39590.3531          +0.0053
       377      -39590.3482          +0.0050
       378      -39590.3436          +0.0046
       379      -39590.3393          +0.0043
       380      -39590.3353          +0.0040
       381      -39590.3314          +0.0038
       382      -39590.3277          +0.0037
       383      -39590.3240          +0.0037
       384      -39590.3204          +0.0037
       385      -39590.3167          +0.0037
       386      -39590.3130          +0.0037
       387      -39590.3092          +0.0038
       388      -39590.3053          +0.0039
       389

GMMHMM(covars_prior=array([[[-1.5, -1.5, -1.5, ..., -1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5, ..., -1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5, ..., -1.5, -1.5, -1.5],
        ...,
        [-1.5, -1.5, -1.5, ..., -1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5, ..., -1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5, ..., -1.5, -1.5, -1.5]],

       [[-1.5, -1.5, -1.5, ..., -1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5, ..., -1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5, ..., -1.5, -1....
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]))

In [41]:
_, path_oy = module.decode(np.array(oy_train_list))
print(f" path of uh is {path_oy}  and path length is {len(path_oy)}")

 path of uh is [0 0 1 ... 0 0 2]  and path length is 3980


In [42]:
phonetic_oy = np.array(uh_train_list)
phonetic_oy

array([[ 0.54857981,  0.66371047, -1.68547961, ...,  1.65189811,
        -1.21632245,  0.55461547],
       [-0.10752098, -0.85489467,  0.29227014, ...,  0.61104407,
        -1.32637077, -1.45740597],
       [-0.69288912, -0.81962079,  0.34783702, ...,  0.01889986,
        -0.93035601, -0.32291001],
       ...,
       [-1.15560746,  0.20256024,  1.4295919 , ..., -1.29409897,
        -0.283016  , -0.95724904],
       [-1.16258076,  0.05581121,  1.14150238, ..., -2.4839669 ,
        -2.40867286, -2.31570041],
       [-1.27422478,  0.26945752,  1.30411541, ..., -2.15263877,
        -2.38354605, -1.79740752]])

In [43]:
phonetic_oy_label = np.array(path_oy).reshape(-1,1)
phonetic_oy_label

array([[0],
       [0],
       [1],
       ...,
       [0],
       [0],
       [2]])

In [44]:
module2.sample(15)

Degenerate mixture covariance


(array([[ 6.18529841e-01,  8.95841657e-01, -1.31142545e+00,
         -6.83435316e-02,  6.18037227e-01,  9.18129204e-02,
         -8.44336025e-01, -1.19764819e+00,  5.35588463e-01,
          3.71746776e+00, -1.36351396e+00,  7.51809255e-01,
          9.17226943e-01],
        [ 1.33782441e-01,  7.74304874e-01, -8.30111322e-01,
         -6.26287832e-01,  5.93879636e-01, -2.67169650e-01,
         -2.87499022e-01, -3.84921898e-02, -1.00848252e+00,
         -2.64536413e-01,  1.26264369e+00,  2.14232918e-02,
         -8.64124790e-01],
        [-5.07029638e-01,  3.38772847e-01, -1.04540633e+00,
         -3.47344374e-01,  6.71433070e-01, -2.83715086e-01,
         -7.76240689e-01, -6.12011794e-01, -1.70401835e+00,
         -2.38875642e-01,  4.35351345e-01,  9.98578221e-02,
         -7.44100999e-01],
        [-2.18599529e-01,  9.92533846e-01,  3.22200227e-01,
          8.09933116e-02, -1.57954913e+00, -3.10478473e+00,
          2.56175432e+00,  4.38687737e-01, -2.19564576e+00,
          7.8349114

# Training GMM-HMM model for 'aw' phoneme

In [45]:
aw_feature_list = list(data[data['labels'] == 'aw']['features'])
aw_feature_list

[array([ 11.60972936,   7.15123147,  -1.60744623, -13.68566927,
        -39.9152273 , -28.28662815,   8.67799017,  -2.87631013,
         -9.96353753, -26.07902674,   7.61391222,  -8.55724165,
         -6.68534632]),
 array([ 11.73238461, -14.06383863,  14.41259321, -11.40044038,
        -28.93646497, -13.19348552,  -2.34445108,  -6.3097387 ,
         -4.14482297, -15.92114227,   2.16289731,   5.35046917,
         -7.59188714]),
 array([  6.97979656, -14.20747608,   4.90735783,  -7.35736961,
          0.46619226,  -2.33434898, -12.3625567 , -20.44019415,
        -26.89977856, -18.33918675,  -8.71098288, -17.11434201,
        -26.54282569]),
 array([  6.64774104, -18.06862378,  10.84326149,   5.14096956,
          6.63927338,  -2.77444325,   1.93733375,   1.55374358,
         -5.23294755,  -6.30769332, -15.77728379, -16.19774078,
        -13.36667517]),
 array([  6.74062952, -28.5561053 ,  -0.83243389,  -4.94928566,
          1.76045622,  -6.05947166,  -6.6021993 ,  -0.54788073,
        

# Normalize 'aw' features with 0 mean and 1 std

In [46]:
temp_data = list()
for j in range(len(aw_feature_list)):
    temp_data.append(np.array(aw_feature_list[j]))
        
        
aw_train_list = list()
for j in range(len(temp_data)):
    aw_train_list.append(((temp_data[j] - np.mean(temp_data, axis=0)) / np.std(temp_data, axis=0)).tolist())

In [47]:
aw_train_list

[[-0.3384908197737182,
  0.7054862776576701,
  0.5122461498737897,
  -0.061197912463589424,
  -1.4423576516315757,
  -1.3715367378294403,
  0.9263757990384406,
  0.3109670402153076,
  -0.35501753894251875,
  -1.4775401783910385,
  0.6398639078090601,
  -0.3495647483464848,
  -0.16809694725977586],
 [-0.30254050219681866,
  -0.4029799514161324,
  1.6180896873944903,
  0.08969355012345984,
  -0.7805675843401727,
  -0.3852972641023328,
  0.21045963743511736,
  0.07422612747434938,
  0.005669833618165303,
  -0.7645473617544399,
  0.17787073803286962,
  0.7953360538292853,
  -0.2516919252866474],
 [-1.6955265247263756,
  -0.4104848647744835,
  0.9619550284620417,
  0.35665361820583175,
  0.9917985469334866,
  0.324277233839333,
  -0.4402241409735274,
  -0.9000935211837388,
  -1.4048522095996345,
  -0.9342724998808135,
  -0.7437298193879038,
  -1.0539963552440097,
  -1.999217522377169],
 [-1.7928521708517169,
  -0.6122259771160113,
  1.3717031268438002,
  1.1819069144052297,
  1.363906408217

In [48]:
gmmhmm_module_list.append(hmm.GMMHMM(n_components=3,n_mix=74,covariance_type='diag',random_state=42,verbose=True,n_iter=500,tol=1e-4))

In [49]:
module3 = gmmhmm_module_list[2]
module3.fit(aw_train_list)

         1      -79170.2084             +nan
         2      -64012.6669      +15157.5415
         3      -58105.9150       +5906.7519
         4      -54998.5973       +3107.3177
         5      -53115.6665       +1882.9308
         6      -51908.9161       +1206.7504
         7      -51099.4695        +809.4465
         8      -50543.4917        +555.9778
         9      -50177.8905        +365.6012
        10      -49933.3605        +244.5300
        11      -49764.6635        +168.6970
        12      -49661.9218        +102.7417
        13      -49586.1429         +75.7789
        14      -49520.1908         +65.9521
        15      -49473.6553         +46.5356
        16      -49437.8890         +35.7662
        17      -49400.8490         +37.0401
        18      -49366.8670         +33.9820
        19      -49344.2749         +22.5921
        20      -49331.0983         +13.1767
        21      -49319.4884         +11.6099
        22      -49307.7977         +11.6906
        23

       184      -48828.8053          +0.0026
       185      -48828.8021          +0.0031
       186      -48828.7983          +0.0038
       187      -48828.7936          +0.0047
       188      -48828.7879          +0.0057
       189      -48828.7810          +0.0069
       190      -48828.7729          +0.0082
       191      -48828.7634          +0.0095
       192      -48828.7526          +0.0108
       193      -48828.7403          +0.0123
       194      -48828.7261          +0.0142
       195      -48828.7090          +0.0171
       196      -48828.6873          +0.0216
       197      -48828.6590          +0.0283
       198      -48828.6213          +0.0377
       199      -48828.5696          +0.0517
       200      -48828.4889          +0.0807
       201      -48828.2849          +0.2041
       202      -48827.2520          +1.0329
       203      -48825.1710          +2.0810
       204      -48824.1270          +1.0440
       205      -48822.9497          +1.1773
       206

GMMHMM(covars_prior=array([[[-1.5, -1.5, -1.5, ..., -1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5, ..., -1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5, ..., -1.5, -1.5, -1.5],
        ...,
        [-1.5, -1.5, -1.5, ..., -1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5, ..., -1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5, ..., -1.5, -1.5, -1.5]],

       [[-1.5, -1.5, -1.5, ..., -1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5, ..., -1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5, ..., -1.5, -1....
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]))

In [50]:
_, path_aw = module.decode(np.array(aw_train_list))
print(f" path of uh is {path_aw}  and path length is {len(path_aw)}")

 path of uh is [0 2 2 ... 2 2 2]  and path length is 4318


In [51]:
phonetic_aw = np.array(aw_train_list)
phonetic_aw.shape

(4318, 13)

In [52]:
phonetic_aw_label = np.array(path_aw).reshape(-1,1)
phonetic_aw_label.shape


(4318, 1)

In [53]:
module3.sample(10)

(array([[-6.91448059e-01,  2.10168098e-01, -1.52636616e-01,
          1.06028892e+00,  9.78277999e-01, -1.19621659e+00,
         -1.52927440e-01, -8.11742101e-02, -3.01411283e-01,
          1.29120402e-01, -6.46501665e-01,  4.68251250e-01,
         -1.24714592e+00],
        [-9.72840611e-01,  2.70155642e-01, -6.01905676e-01,
          7.70241143e-01,  7.59317992e-01, -3.43723196e-01,
         -9.46965592e-02, -6.84994368e-01, -1.08153007e+00,
         -6.87272081e-01, -1.68446080e-03,  1.08956931e-01,
         -1.11875661e+00],
        [ 3.24216246e-01,  4.02578317e-02, -1.29853665e+00,
          5.66643473e-01, -9.23174286e-01, -4.56517430e-01,
         -1.64246953e+00, -8.24895855e-01, -3.13899359e-02,
          6.85700902e-01, -3.54962846e-01, -1.40712724e+00,
         -5.48204949e-01],
        [-1.27848677e-01,  7.66140913e-01,  8.11050172e-01,
         -1.86641641e-01,  3.92257573e-02, -2.03414599e+00,
         -8.18437301e-01, -4.66352347e-01, -4.54714548e-02,
         -1.8825969

# Training GMM-HMM model for 'th' phoneme

In [54]:
th_feature_list = list(data[data['labels'] == 'th']['features'])
th_feature_list

[array([ 14.28669405,   5.76446812, -33.8388639 , -23.7384949 ,
        -30.15383126, -12.90573806, -28.43012209,   0.22460137,
         14.89896977, -17.95077675,   3.92641412, -14.34615805,
        -15.27826363]),
 array([ 14.31664306,   3.12851862, -21.32918759, -13.99003876,
        -17.24065302,  -4.80162781, -12.68990661,   9.67022948,
         21.49401489,  -7.54592868,   0.45233457,  -7.53382742,
         -9.91976239]),
 array([ 14.72219336,   7.97168209, -40.90885644, -14.3134532 ,
        -33.5802708 ,  -4.0674944 , -22.18065283,   5.82579908,
         24.99866731, -19.53117747,   9.80931309, -14.72117151,
         -9.04655435]),
 array([ 14.58167643,   9.49787631, -38.29119646,  -9.02598314,
        -28.79272397,   1.20797986, -15.14632353,   9.75398775,
         24.68096675, -16.95363271,   8.78393575, -14.38352437,
         -8.80164387]),
 array([ 14.86351883,   8.3117467 , -45.58485754, -11.65855669,
        -35.19563402,  -2.91571268, -20.01724617,   7.24045129,
        

# Normalize 'th' features with 0 mean and 1 std

In [55]:
temp_data = list()
for j in range(len(th_feature_list)):
    temp_data.append(np.array(th_feature_list[j]))
        
        
th_train_list = list()
for j in range(len(temp_data)):
    th_train_list.append(((temp_data[j] - np.mean(temp_data, axis=0)) / np.std(temp_data, axis=0)).tolist())

In [56]:
np.shape(th_train_list)

(5495, 13)

In [57]:
gmmhmm_module_list.append(hmm.GMMHMM(n_components=3,n_mix=95,covariance_type='diag',random_state=42,verbose=True,n_iter=500,tol=1e-4))

In [58]:
module4 = gmmhmm_module_list[3]
module4.fit(th_train_list)

         1     -101820.2700             +nan
         2      -78757.5057      +23062.7643
         3      -70124.7138       +8632.7919
         4      -65215.0297       +4909.6840
         5      -62129.8452       +3085.1845
         6      -60467.7223       +1662.1229
         7      -59479.8017        +987.9206
         8      -58791.2386        +688.5631
         9      -58000.2751        +790.9635
        10      -53507.5187       +4492.7564
        11      -53256.3175        +251.2012
        12      -53102.5936        +153.7238
        13      -53002.7386         +99.8551
        14      -52935.0304         +67.7082
        15      -52883.3611         +51.6693
        16      -52819.5724         +63.7887
        17      -52771.8019         +47.7705
        18      -52743.5987         +28.2032
        19      -52718.5628         +25.0359
        20      -52686.6696         +31.8932
        21      -52661.1791         +25.4905
        22      -52644.3697         +16.8094
        23

GMMHMM(covars_prior=array([[[-1.5, -1.5, -1.5, ..., -1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5, ..., -1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5, ..., -1.5, -1.5, -1.5],
        ...,
        [-1.5, -1.5, -1.5, ..., -1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5, ..., -1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5, ..., -1.5, -1.5, -1.5]],

       [[-1.5, -1.5, -1.5, ..., -1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5, ..., -1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5, ..., -1.5, -1....
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]))

In [59]:
_, path_th = module.decode(np.array(th_train_list))
print(f" path of uh is {path_th}  and path length is {len(path_th)}")

 path of uh is [0 1 0 ... 0 0 0]  and path length is 5495


In [60]:
phonetic_th = np.array(th_train_list)
phonetic_th

array([[ 0.15219354,  0.76428024, -1.41343388, ...,  0.4153344 ,
        -0.99188517, -1.11553372],
       [ 0.16111699,  0.6066002 , -0.44824905, ...,  0.12573745,
        -0.41491976, -0.61542902],
       [ 0.28195261,  0.89631371, -1.95891958, ...,  0.90572893,
        -1.02364666, -0.53393321],
       ...,
       [ 0.01046151,  1.26480096,  0.51505735, ..., -1.33873987,
        -0.04915353,  1.04898728],
       [ 0.2079255 ,  1.77322125,  0.61341412, ..., -1.57631453,
         0.35783299,  1.49484736],
       [ 0.22904552,  1.87375357,  0.75966672, ..., -1.00216017,
         0.2192726 ,  1.36075928]])

In [61]:
phonetic_th_label = np.array(path_th).reshape(-1,1)
phonetic_th_label

array([[0],
       [1],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [62]:
ch_feature_list = list(data[data['labels'] == 'ch']['features'])
ch_feature_list

[array([ 14.75046478,  -5.58408298, -22.00283827,   5.41701546,
        -39.92062886, -29.97225132, -56.2334252 , -13.63297719,
        -20.83134637, -32.04460536,  21.19864722, -15.2449542 ,
          1.26157813]),
 array([ 14.92638929,   0.31788166, -18.81455146,  19.21304704,
        -34.41720174, -15.09062011, -39.2707877 ,   5.36788197,
         -0.60835612, -22.26768277,  36.23747944, -16.65117459,
          2.71003656]),
 array([ 14.64102535,  -2.8267306 , -20.9621227 ,  15.80006815,
        -33.67409051, -20.66983733, -36.32918724,   8.28690343,
         -4.01478954, -33.28729194,  32.03855604, -25.66907568,
         -7.33197564]),
 array([ 14.07890748,  -3.20353721, -17.65532165,  18.90170934,
        -30.95472017, -16.34675287, -25.913505  ,  10.84013992,
         -9.04956376, -47.80231678,  35.45695616, -31.59237363,
        -12.5596023 ]),
 array([ 15.46645906,   2.31257662, -30.7148671 , -24.18459244,
        -23.96103587, -13.59951201, -12.29340218, -13.93293256,
        

In [63]:
temp_data = list()
for j in range(len(ch_feature_list)):
    temp_data.append(np.array(ch_feature_list[j]))
        
        
ch_train_list = list()
for j in range(len(temp_data)):
    ch_train_list.append(((temp_data[j] - np.mean(temp_data, axis=0)) / np.std(temp_data, axis=0)).tolist())

In [64]:
gmmhmm_module_list.append(hmm.GMMHMM(n_components=3,n_mix=128,covariance_type='diag',random_state=42,verbose=True,n_iter=600,tol=1e-4))

In [65]:
module5 = gmmhmm_module_list[4]
module5.fit(ch_train_list)

         1     -135992.9464             +nan
         2     -108268.2760      +27724.6703
         3      -97770.1414      +10498.1346
         4      -91184.5806       +6585.5609
         5      -86423.5626       +4761.0180
         6      -83211.9128       +3211.6498
         7      -81181.3754       +2030.5374
         8      -79929.0793       +1252.2961
         9      -79107.1856        +821.8937
        10      -78555.1062        +552.0793
        11      -78190.8135        +364.2928
        12      -77904.5769        +286.2366
        13      -77707.9263        +196.6506
        14      -77576.8003        +131.1260
        15      -77477.7421         +99.0582
        16      -77403.9031         +73.8390
        17      -77340.6903         +63.2128
        18      -77274.8899         +65.8004
        19      -77206.3547         +68.5352
        20      -77138.0261         +68.3286
        21      -77109.8796         +28.1465
        22      -77088.5176         +21.3620
        23

       184      -76805.5699          +0.0048
       185      -76805.5626          +0.0074
       186      -76805.5504          +0.0122
       187      -76805.5287          +0.0217
       188      -76805.4881          +0.0406
       189      -76805.4123          +0.0758
       190      -76805.2804          +0.1319
       191      -76805.0719          +0.2085
       192      -76804.7615          +0.3104
       193      -76804.3976          +0.3639
       194      -76804.1199          +0.2777
       195      -76803.9567          +0.1632
       196      -76803.8902          +0.0665
       197      -76803.8733          +0.0169
       198      -76803.8701          +0.0033
       199      -76803.8695          +0.0006
       200      -76803.8694          +0.0001
       201      -76803.8693          +0.0000


GMMHMM(covars_prior=array([[[-1.5, -1.5, -1.5, ..., -1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5, ..., -1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5, ..., -1.5, -1.5, -1.5],
        ...,
        [-1.5, -1.5, -1.5, ..., -1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5, ..., -1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5, ..., -1.5, -1.5, -1.5]],

       [[-1.5, -1.5, -1.5, ..., -1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5, ..., -1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5, ..., -1.5, -1....
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]))

In [66]:
_, path_ch = module.decode(np.array(ch_train_list))
print(f" path of uh is {path_ch}  and path length is {len(path_ch)}")

 path of uh is [0 2 2 ... 2 2 2]  and path length is 7426


In [67]:
phonetic_ch = np.array(ch_train_list)
phonetic_ch

array([[ 0.18817161, -0.01486208, -0.27703263, ...,  1.78135024,
        -1.00443981,  0.37486897],
       [ 0.24636679,  0.35644689, -0.05737938, ...,  2.93565224,
        -1.11529517,  0.50867764],
       [ 0.15196946,  0.15861061, -0.20533376, ...,  2.61336487,
        -1.82619557, -0.41900396],
       ...,
       [-2.46949311, -1.58799498,  1.10033944, ..., -0.58749041,
        -0.67932666, -0.86898124],
       [-2.45508622, -1.52611008,  1.04147825, ..., -0.61123747,
        -0.69048329, -1.1235622 ],
       [-0.94723067, -1.79016968,  1.20172882, ...,  0.17430367,
         0.132899  ,  0.39227996]])

In [68]:
phonetic_ch_label = np.array(path_ch).reshape(-1,1)
phonetic_ch_label

array([[0],
       [2],
       [2],
       ...,
       [2],
       [2],
       [2]])